In [ ]:
import sys, time, os, asyncio, glob

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from astropy.time import Time, TimeDelta

from lsst_efd_client import EfdClient
from lsst.daf.butler import Butler

In [ ]:
client = EfdClient('summit_efd')
butler = Butler('/repo/LATISS', collections="LATISS/raw/all")

In [ ]:
max_slopes = [[2022110300020, 212],
 [2022110800013, 205],
 [2022110900005, 181],
 [2022111000500, 143],
 [2022111000600, 140]]

In [ ]:
shift_index = []
reb_temps = []
expIds = []
for [expId, max_slope] in max_slopes:
    mData = butler.get('raw.metadata', detector=0, exposure=expId)
    start = Time(mData['DATE-BEG']) - TimeDelta(30.0, format='sec')
    end = Time(mData['DATE-END']) + TimeDelta(30.0, format='sec')
    reb_temp_data = await client.select_time_series("lsst.sal.ATCamera.focal_plane_Reb", ["temp20"], start, end)
    reb_temp = reb_temp_data.values[0][0]
    shift_index.append(max_slope)
    reb_temps.append(reb_temp)
    expIds.append(expId)
    

In [ ]:
plt.figure(figsize=(8,8))
plt.title("AuxTel bias shift vs WREB temperature")
plt.scatter(reb_temps, shift_index)
for i, expId in enumerate(expIds):
    plt.text(reb_temps[i] + 0.5, shift_index[i], f"{expId}")
plt.ylabel("Bias wave max slope index")
plt.xlabel("WREB temp (C)")
xplot = np.linspace(15,35,100)
fit = np.polyfit(reb_temps, shift_index, 1)
yplot = fit[0] * xplot + fit[1]
plt.plot(xplot, yplot, ls = '--', color='green')
                    
plt.xlim(15, 35)
plt.savefig(f"/home/craiglagegit/DATA/AuxTel_Bias_Shift_vs_Temperature_28Nov22.pdf")

In [ ]:
plt.figure(figsize=(16,8))
plt.suptitle("AuxTel bias shift vs WREB temperature", fontsize=24)
plt.subplots_adjust(wspace=0.5)

temps = ['temp10', 'temp20', 'temp30', 'temp40', 'temp50', 'temp60', 'aspicl_Temp00', 'aspicu_Temp00']
for i, temp in enumerate(temps):
    shift_index = []
    reb_temps = []
    expIds = []
    for [expId, max_slope] in max_slopes:
        mData = butler.get('raw.metadata', detector=0, exposure=expId)
        start = Time(mData['DATE-BEG']) - TimeDelta(30.0, format='sec')
        end = Time(mData['DATE-END']) + TimeDelta(30.0, format='sec')
        reb_temp_data = await client.select_time_series("lsst.sal.ATCamera.focal_plane_Reb", [temp], start, end)
        reb_temp = reb_temp_data.values[0][0]
        shift_index.append(max_slope)
        reb_temps.append(reb_temp)
        expIds.append(expId)
    plt.subplot(2,4,i+1)
    plt.scatter(reb_temps, shift_index)
    for i, expId in enumerate(expIds):
        plt.text(reb_temps[i] + 0.5, shift_index[i], f"{expId}", fontsize=6)
    plt.ylabel("Bias wave max slope index")
    plt.xlabel(f"WREB temp {temp} (C)")
    xplot = np.linspace(0,35,100)
    fit = np.polyfit(reb_temps, shift_index, 1)
    yplot = fit[0] * xplot + fit[1]
    plt.plot(xplot, yplot, ls = '--', color='green')
    plt.xlim(0, 35)
plt.savefig(f"/home/craiglagegit/DATA/AuxTel_Bias_Shift_vs_Temperature_30Nov22.pdf")    